In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
import os
import torch

In [46]:
# Load the training data
train_data = pd.read_csv('../../2nd-Comp-Data/train.csv')
test_data = pd.read_csv('../../2nd-Comp-Data/test.csv')

used = []

# Extract features and target variable
X = train_data.drop('price_doc', axis=1)
y = train_data['price_doc']
X_test = test_data.drop(['row ID'], axis=1)

In [47]:
# X = pd.get_dummies(X)
# X_test = pd.get_dummies(X_test) 

In [48]:
# drop all columns in X_train with dtypes object
for col in X.columns:
    if X[col].dtype == 'object':
        X.drop(col, axis=1, inplace=True)

# drop all columns in X_test with dtypes object
for col in X_test.columns:
    if X_test[col].dtype == 'object':
        X_test.drop(col, axis=1, inplace=True)

used.append("Removed Object Dtypes")

In [49]:
X = X.astype('float32')
X_test = X_test.astype('float32')

In [50]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [51]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
used.append("StandardScaler")

In [52]:
os.environ["KERAS_BACKEND"] = "torch"
used.append("Keras With Torch Backend")

In [53]:
features_used = X_train.shape[1]

# Build the neural network
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.3))  # You can use either dropout or early stopping
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='linear'))  # Output layer with linear activation for regression

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=1800, validation_data=(X_val, y_val), callbacks=[early_stopping])

used.append("2 Hidden Layers: Hidden Layer 1 with 128 neurons relu activation\n, Hidden Layer 2 with 64 neurons relu activation, Dropout 0.3, O linear")
used.append("Adam Optimizer")
used.append("Loss Calculation: Mean Squared Error")
used.append("EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)")
used.append("batch_size=1800, epochs=100, early_stopping, dropout=0.3 (used in hidden layers)")

Epoch 1/100
81/81 [==============================] - 2s 22ms/step - loss: 692468249001984.0000 - val_loss: 701411310436352.0000
Epoch 2/100
81/81 [==============================] - 1s 16ms/step - loss: 692244642267136.0000 - val_loss: 700921952600064.0000
Epoch 3/100
81/81 [==============================] - 1s 11ms/step - loss: 691306795892736.0000 - val_loss: 699395494379520.0000
Epoch 4/100
81/81 [==============================] - 1s 11ms/step - loss: 689040663773184.0000 - val_loss: 696238492090368.0000
Epoch 5/100
81/81 [==============================] - 1s 14ms/step - loss: 684888571248640.0000 - val_loss: 690936757616640.0000
Epoch 6/100
81/81 [==============================] - 1s 18ms/step - loss: 678399546753024.0000 - val_loss: 683022139523072.0000
Epoch 7/100
81/81 [==============================] - 1s 14ms/step - loss: 669111747084288.0000 - val_loss: 672150973317120.0000
Epoch 8/100
81/81 [==============================] - 1s 11ms/step - loss: 656872164032512.0000 - val_los

In [54]:
# Make predictions
predictions = model.predict(X_test)

# Create a DataFrame with the results
submission_df = pd.DataFrame({'row ID': test_data['row ID'], 'price_doc': predictions.flatten()})

# Save the results to a CSV file
submission_df.to_csv('Day7.1.csv', index=False)

2431/2431 [==============================] - 2s 798us/step


In [55]:
print("Features Used = " + str(features_used) + "\n")
for i in used:
    print(i)

Features Used = 256

Removed Object Dtypes
StandardScaler
Keras With Torch Backend
Layers: H1 128 relu, H2 64 relu, Dropout 0.3, O linear
Compiled with loss='mean_squared_error', optimizer='adam'
EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
epochs=100, batch_size=1800
